# Proyecto Final
## Procesamiento de Datos
##### Gonzalo Fernández Suárez

---

1) Crear un CALLER para descargar noticias de Cryptomonedas (2.0 puntos de 10)

1.1 - Conectarse a la API de twitters para descargar tweets sobre Cryptomonedas (0.5 puntos de 10).

1.2 - Conectarse a la API de Gdelt para descargar noticias sobre Cryptomonedas (0.5 puntos de 10).

1.3 - Scrapear la web https://coinmarketcal.com/en/news para descargar noticicas sobre Cryptomonedas (1.0 puntos de 10).

2) Crear un SERVER de Spark Streamming (1 puntos de 10)

3) Hacer un analisis de los mensajes en directo en clase, con ventana de tiempo (1 puntos de 10)

4) Aplicar un modelo de analisis de sentimiento (2.5 puntos de 10)

4.1 - Aplicar uno de los modelo como Vader Sentiment Analysis, textblob, flair o FinBERT (+0.5 puntos de 10).

4.2 - Aplicar todos los modelo: Vader Sentiment Analysis, textblob, flair y FinBERT (+0.5 puntos de 10).

4.3 - Crear un modelo de machine learning propio y aplicar dicho modelo a los mensajes (+1.5 puntos de 10).

Utilizando los siguientes datos etiquetados manualmente:
20220421_FULLDATASET_FOR_UTAD.csv

4.3.1 - Utilizar news como texto y final_manual_labelling como variable target para vuestro modelo!

 

5) Guardar los datos del SERVER en SQL marcados por tiempo (1.0 puntos de 10)

6) Presentar gráficas de evolución de los sentimiento y comparativa (1.5 puntos de 10)

6.1 - Presentar gráficas de evolución de los sentimiento, uno de los modelo como Vader Sentiment Analysis, textblob y flair (+1.0 puntos de 10).

6.2 - Presentar gráficas de evolución de los sentimiento de todos los modelo: Vader Sentiment Analysis, textblob y flair (+0.5 puntos de 10).

7) Incluir en la comparativa tu propio modelo, y ganas nota máxima si tu modelo es mejeor que los anteriores (+1.0 puntos de 10).

[Tutorial instalacion spark](https://phoenixnap.com/kb/install-spark-on-windows-10).
El link de Java esta mal, descargar JDK no JRE

[Runear spark](https://aamargajbhiye.medium.com/apache-spark-setup-a-multi-node-standalone-cluster-on-windows-63d413296971)
```
cmd >> cd C:\Users\User\Desktop\U-Tad\Procesamiento de Datos\spark-3.2.1-bin-hadoop3.2

# Levantar Master Node
cmd >> bin\spark-class org.apache.spark.deploy.master.Master --host localhost
```
http://DESKTOP-GFS:8080

```
# Poner a escuchar al Worker Node
cmd >> bin\spark-class org.apache.spark.deploy.worker.Worker spark://localhost:7077 --host localhost
```

# Aplicacion PYTHON

Nota: Este proyecto está desarrollado enteramente en un sistema **Windows**. Las máquinas virtuales Ubuntu me han dado malas experiencias petandome el ordenador o bien rompiendose dañando toda la informacion que habia en ellas.

## 1.3 Scrapear la web https://coinmarketcal.com/en/news para descargar noticicas sobre Cryptomonedas

## 2 Crear un SERVER de Spark Streamming

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [4]:
# Inicializamos el Contexto
sc = SparkContext("local[2]" , "NetworkWordCount" ) # Master y nombre de la app
ssc = StreamingContext(sc, 10) # Contexto de streaming, contexto de spark y cada cuanto se repite el proceso

# Inicializamos el socket => Comunicamos el origen de los datos
lines = ssc.socketTextStream("localhost", 9090) # Host, Puerto

## 3 Hacer un analisis de los mensajes en directo, con ventana de tiempo

Retocar comentarios del siguiente bloque de codigo

In [5]:
# Como vamos a procesar los datos
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word,1))
wordCounts = pairs.reduceByKey(lambda x,y: x+y)

wordCounts.pprint()

Ahora en la consola de comandos nos conectamos, usando netCat, a "localhost" en el puerto 9090. 

[Instalación netCat en Windows](https://programmerclick.com/article/5414801604/)

Abrimos la consola de comandos y ejecutamos:
```
cmd >> nc -L -p 9090    # Nos conectamos con el escuchador del puerto 9090
```
Ahora desde la cmd podemos enviar información en tiempo real a Spark para que la analice.

In [6]:
ssc.start() # Iniciamos el Streaming Context
ssc.awaitTermination()

-------------------------------------------
Time: 2022-05-01 18:23:20
-------------------------------------------
('Hola', 1)

-------------------------------------------
Time: 2022-05-01 18:23:30
-------------------------------------------
('Soy', 1)
('Gonzalo', 1)
('probando', 1)
('Estoy', 1)

-------------------------------------------
Time: 2022-05-01 18:23:40
-------------------------------------------
('como', 1)
('Spark', 1)
('funciona', 1)
('Streaming', 1)

-------------------------------------------
Time: 2022-05-01 18:23:50
-------------------------------------------
('mensajes', 1)
('en', 1)
('Con', 1)
('directo', 1)

-------------------------------------------
Time: 2022-05-01 18:24:00
-------------------------------------------

-------------------------------------------
Time: 2022-05-01 18:24:10
-------------------------------------------

-------------------------------------------
Time: 2022-05-01 18:24:20
-------------------------------------------



In [ ]:
# Paramos el Streaming Context
ssc.stop()
# Finalizamos el Spark Context
sc.stop()

## 4 Aplicar un modelo de analisis de sentimiento a los mensajes

### 4.1 Aplicar uno de los modelo como Vader Sentiment Analysis, textblob, flair o FinBERT